In this competetition, our goal is to predict pawpularity of pet's profile based on appearance of pet profile. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
tran=pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test=pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [ ]:
column=[col for col in tran.columns if col not in ['Id','Pawpularity']]

In [ ]:
x=tran[column]
y=tran['Pawpularity']

In [ ]:
print(x.shape,y.shape)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV,KFold
from sklearn.metrics import mean_squared_error
xtrain,xtest,ytrain,ytest=train_test_split(x,y,random_state=42,test_size=0.2)

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
cat=CatBoostRegressor(random_state=42,eval_metric='RMSE')

In [ ]:
kfold=KFold(n_splits=5,shuffle=True,random_state=42)

In [ ]:
params={'learning_rate': [0.01,0.1,0.05],
       'depth':[8,10,15],
       'iterations':[10,30,50],
    }

In [ ]:
grid_cat=GridSearchCV(estimator=cat,param_grid=params,cv=kfold)

In [ ]:
grid_cat=grid_cat.fit(xtrain,ytrain,verbose=0)

In [ ]:
grid_cat.best_params_

In [ ]:
grid_cat.best_score_

In [ ]:
cat1=CatBoostRegressor(**grid_cat.best_params_)

In [ ]:
cat1.fit(xtrain,ytrain,verbose=0)

In [ ]:
y_pred=cat1.predict(xtest)

In [ ]:
np.sqrt(mean_squared_error(ytest,y_pred))

In [ ]:
test_id=test['Id']
test.drop('Id',inplace=True,axis=1)

In [ ]:
sub=cat1.predict(test)

In [ ]:
submission=pd.DataFrame(sub,columns=['Pawpularity'])
submission=pd.concat([test_id,submission],axis=1)

In [ ]:
submission.to_csv('submission.csv',index=False)